In [1]:
# Milestone 2: Instruction Parser → Action Mapping → LangGraph Workflow
# Project: AI Agent for Automated Website Testing
# Author: <Your Name>
# Purpose: Convert natural language test instructions into structured actions


In [2]:
!pip install langgraph langchain-core


In [3]:
from typing import TypedDict, List
from datetime import datetime
from langgraph.graph import StateGraph
import json


In [4]:
class AgentState(TypedDict):
    instruction: str
    actions: list
    timeline: list
    generated_code: str


In [5]:
class InstructionParser:
    def parse(self, instruction: str):
        steps = instruction.split(",")
        actions = []

        for i, step in enumerate(steps):
            step = step.strip().lower()

            if "open" in step:
                action = "open"
                target = "home_page"
            elif "click" in step:
                action = "click"
                target = "login_button"
            elif "type" in step:
                action = "fill"
                target = "credentials"
            elif "submit" in step:
                action = "submit"
                target = "login_form"
            elif "assert" in step:
                action = "assert"
                target = "login_success"
            else:
                action = "unknown"
                target = step

            actions.append({
                "step": i + 1,
                "action": action,
                "target": target,
                "confidence": round(0.75 + i * 0.05, 2)
            })

        return actions


In [6]:
class Timeline:
    def __init__(self):
        self.events = []

    def record(self, event):
        self.events.append({
            "event": event,
            "timestamp": datetime.now().isoformat()
        })

    def get(self):
        return self.events


In [7]:
class CodeGenerator:
    def generate(self, actions):
        lines = []
        lines.append("def run_test():")
        lines.append("    logs = []")

        for a in actions:
            if a["action"] == "open":
                lines.append("    logs.append('Opening home page')")
            elif a["action"] == "click":
                lines.append("    logs.append('Clicking login button')")
            elif a["action"] == "fill":
                lines.append("    logs.append('Filling username and password')")
            elif a["action"] == "submit":
                lines.append("    logs.append('Submitting form')")
            elif a["action"] == "assert":
                lines.append("    logs.append('Asserting login success')")

        lines.append("    return logs")
        return "\n".join(lines)


In [8]:
parser = InstructionParser()
generator = CodeGenerator()

def parser_node(state: AgentState):
    actions = parser.parse(state["instruction"])
    return {"actions": actions}

def timeline_node(state: AgentState):
    tl = Timeline()
    for a in state["actions"]:
        tl.record(f"{a['action']} → {a['target']}")
    return {"timeline": tl.get()}

def generator_node(state: AgentState):
    code = generator.generate(state["actions"])
    return {"generated_code": code}


In [9]:
workflow = StateGraph(AgentState)

workflow.add_node("parse", parser_node)
workflow.add_node("timeline", timeline_node)
workflow.add_node("generate", generator_node)

workflow.set_entry_point("parse")
workflow.add_edge("parse", "timeline")
workflow.add_edge("timeline", "generate")

app = workflow.compile()


In [10]:
instruction = "Open home page, click login, type username and password, submit, assert login success"

result = app.invoke({
    "instruction": instruction
})


In [11]:
print("=== STRUCTURED ACTIONS ===")
print(json.dumps(result["actions"], indent=2))


=== STRUCTURED ACTIONS ===
[
  {
    "step": 1,
    "action": "open",
    "target": "home_page",
    "confidence": 0.75
  },
  {
    "step": 2,
    "action": "click",
    "target": "login_button",
    "confidence": 0.8
  },
  {
    "step": 3,
    "action": "fill",
    "target": "credentials",
    "confidence": 0.85
  },
  {
    "step": 4,
    "action": "submit",
    "target": "login_form",
    "confidence": 0.9
  },
  {
    "step": 5,
    "action": "assert",
    "target": "login_success",
    "confidence": 0.95
  }
]


In [12]:
print("\n=== ACTION TIMELINE ===")
print(json.dumps(result["timeline"], indent=2))



=== ACTION TIMELINE ===
[
  {
    "event": "open \u2192 home_page",
    "timestamp": "2025-12-21T03:46:18.515466"
  },
  {
    "event": "click \u2192 login_button",
    "timestamp": "2025-12-21T03:46:18.515475"
  },
  {
    "event": "fill \u2192 credentials",
    "timestamp": "2025-12-21T03:46:18.515479"
  },
  {
    "event": "submit \u2192 login_form",
    "timestamp": "2025-12-21T03:46:18.515481"
  },
  {
    "event": "assert \u2192 login_success",
    "timestamp": "2025-12-21T03:46:18.515484"
  }
]


In [13]:
print("\n=== GENERATED TEST CODE ===")
print(result["generated_code"])



=== GENERATED TEST CODE ===
def run_test():
    logs = []
    logs.append('Opening home page')
    logs.append('Clicking login button')
    logs.append('Filling username and password')
    logs.append('Submitting form')
    logs.append('Asserting login success')
    return logs


In [14]:
local_env = {}
exec(result["generated_code"], {}, local_env)

logs = local_env["run_test"]()
print("\n=== EXECUTION LOGS ===")
for log in logs:
    print(log)



=== EXECUTION LOGS ===
Opening home page
Clicking login button
Filling username and password
Submitting form
Asserting login success
